In [13]:
%pip install -U langchain-community faiss-cpu langchain-huggingface pymupdf tiktoken langchain-ollama python-dotenv

INFO: pip is looking at multiple versions of scipy to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/13.8 MB ? eta -:--:--
   --- ------------------------------------ 1.0/13.8 MB 6.3 MB/s eta 0:00:03
   ----------- ---------------------------- 3.9/13.8 MB 10.7 MB/s eta 0:00:01
   ------------------- -------------------- 6.8/13.8 MB 11.7 MB/s eta 0:00:01
   ---------------------------- ----------- 9.7/13.8 MB 12.3 MB/s eta 0:00:01
   ------------------------------------ --- 12.6/13.8 MB 12.5 MB/s eta 0:00:01
   ---------------------------------------  13.6/13.8 MB 11.4 MB/s eta 0:00:01
   ---------------------------------------- 13.8/13.8 MB 10.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/16.6 MB ? eta -:--:--
   -- ------------------------------------- 1.0/16.6 MB 8.4 MB/s eta 0:00:02
   ----- ---------------------------------- 2.1/16.6 MB 5.9 MB/s eta 0:00:03
   ----- -------

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.13.0 requires numpy<=1.24.3,>=1.22, but you have numpy 1.26.4 which is incompatible.
tensorflow-intel 2.13.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.2 which is incompatible.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.12.2 which is incompatible.


In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\91939\Downloads\newsai\Scripts\python.exe -m pip install --upgrade pip' command.


In [27]:
import pandas as pd
news_data = pd.read_csv(r"C:\Users\91939\Downloads\Sunil_news.csv")
news_texts = news_data['News'].dropna().tolist()


In [28]:
news_texts[0]

'Pune: Minutes after attending a public rally with Telangana chief minister Revanth Reddy at Ghorpadi, Ramesh Bagwe is ready for another voter-outreach initiative. It\'s the last day of campaign and he cannot afford to waste a minute. Soon, he joins a group of enthusiastic supporters waiting on their bikes near his office on Mahatma Gandhi Road in Pune Cantonment assembly constituency. As he appeals to voters while crossing Sarbatwala Chowk, his wife, Zainab, accompanies him in an open black SUV. An autorickshaw with a person making an announcement of his name and symbol leads the bike rally as it snakes through Chhatrapati Shivaji Market, the 139-year-old British-era structure. The former state minister and also Congress nominee aims to regain Pune Cantonment, a party stronghold between 1999 and 2014, before BJP snatched it following a Narendra Modi wave and retained it five years later. The seat is reserved for scheduled caste candidates. "Bike rallies were planned for the whole day 

In [4]:
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain_ollama import ChatOllama 
from langchain import hub
from langchain_ollama import OllamaEmbeddings
import faiss
from langchain_community.vectorstores import FAISS 
from langchain_community.docstore.in_memory import InMemoryDocstore

In [29]:
documents = [Document(page_content=text) for text in news_texts]
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
news_chunks = text_splitter.split_documents(documents)

In [30]:
news_chunks

[Document(metadata={}, page_content='Pune: Minutes after attending a public rally with Telangana chief minister Revanth Reddy at Ghorpadi, Ramesh Bagwe is ready for another voter-outreach initiative. It\'s the last day of campaign and he cannot afford to waste a minute. Soon, he joins a group of enthusiastic supporters waiting on their bikes near his office on Mahatma Gandhi Road in Pune Cantonment assembly constituency. As he appeals to voters while crossing Sarbatwala Chowk, his wife, Zainab, accompanies him in an open black SUV. An autorickshaw with a person making an announcement of his name and symbol leads the bike rally as it snakes through Chhatrapati Shivaji Market, the 139-year-old British-era structure. The former state minister and also Congress nominee aims to regain Pune Cantonment, a party stronghold between 1999 and 2014, before BJP snatched it following a Narendra Modi wave and retained it five years later. The seat is reserved for scheduled caste candidates. "Bike ral

In [31]:
len(news_chunks)

81

In [32]:
embeddings = OllamaEmbeddings(model='nomic-embed-text', base_url='http://localhost:11434')

vector_store = FAISS.from_documents(documents=news_chunks, embedding=embeddings)

# vector_store = Chroma.from_documents(news_chunks, embeddings)

In [33]:
question = "who is Ramesh bagwe?"
docs = vector_store.search(query=question, k=5, search_type="similarity")

In [34]:
docs

[Document(id='67dbb3a8-2212-407b-92a9-9cf1133fb148', metadata={}, page_content="Wardha: Deoli police on Sunday registered non-cognizable offence against Congress MLA from Deoli assembly constituency, Ranjeet Kamble, on charges of insulting and threatening a gram panchayat (GP) member. PI Bhanudas Pidurkar said a case was lodged under Sections 323, 504 and 506 of IPC against Ranjeet Kamble after receiving a complaint from GP member Ganesh Satav of Senegaon (Bai). Poll Do you support Gurdwara panel's demand to rename Delhi's IGI Airport to Guru Tegh Bahadur Airport? No Yes He said that Senegaon (Bai) sarpanch Ashwini Adkine had filed a complaint against Satav for allegedly insulting Kamble on Thursday during a function in the village. Addressing a press conference, Satav said he questioned Kamble over organizing groundbreaking ceremonies for public works and installing a signboard. “But work is never done. Every citizen should ask such questions to people’s representatives. When I asked 

In [35]:

retriever = vector_store.as_retriever(search_type = 'similarity', 
                                      search_kwargs = {'k': 3})

retriever.invoke(question)

[Document(id='67dbb3a8-2212-407b-92a9-9cf1133fb148', metadata={}, page_content="Wardha: Deoli police on Sunday registered non-cognizable offence against Congress MLA from Deoli assembly constituency, Ranjeet Kamble, on charges of insulting and threatening a gram panchayat (GP) member. PI Bhanudas Pidurkar said a case was lodged under Sections 323, 504 and 506 of IPC against Ranjeet Kamble after receiving a complaint from GP member Ganesh Satav of Senegaon (Bai). Poll Do you support Gurdwara panel's demand to rename Delhi's IGI Airport to Guru Tegh Bahadur Airport? No Yes He said that Senegaon (Bai) sarpanch Ashwini Adkine had filed a complaint against Satav for allegedly insulting Kamble on Thursday during a function in the village. Addressing a press conference, Satav said he questioned Kamble over organizing groundbreaking ceremonies for public works and installing a signboard. “But work is never done. Every citizen should ask such questions to people’s representatives. When I asked 

In [36]:
prompt = hub.pull("rlm/rag-prompt")
prompt

c:\Users\91939\Downloads\newsai\lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [37]:
prompt = """
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    Answer in bullet points. Make sure your answer is relevant to the question and it is answered from the context only.
    Question: {question} 
    Context: {context} 
    Answer:
"""

prompt = ChatPromptTemplate.from_template(prompt)

In [25]:
pip install torch

In [3]:
import torch

In [ ]:
torch.cuda.empty_cache()

In [38]:
llm = ChatOllama(model='mistral', base_url='http://localhost:11434')
llm.invoke('hi')

AIMessage(content=" Hello! How can I help you today? If you have any questions or need assistance with something, feel free to ask. I'm here to help!\n\nFor example, if you want to know something about a specific topic, just let me know what it is and I can provide some information on that subject. Or perhaps you need help with a problem or project? I'll do my best to assist you in any way I can. So go ahead, ask away! :)", additional_kwargs={}, response_metadata={'model': 'mistral', 'created_at': '2025-01-06T18:20:02.2834446Z', 'done': True, 'done_reason': 'stop', 'total_duration': 30576199600, 'load_duration': 8522023300, 'prompt_eval_count': 6, 'prompt_eval_duration': 3441000000, 'eval_count': 100, 'eval_duration': 18593000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-7eb5db9f-bbc0-46d8-ad12-448140b7c797-0', usage_metadata={'input_tokens': 6, 'output_tokens': 100, 'total_tokens': 106})

In [39]:

def format_docs(docs):
    return '\n\n'.join([doc.page_content for doc in docs])

context = format_docs(docs)
print(context)

Wardha: Deoli police on Sunday registered non-cognizable offence against Congress MLA from Deoli assembly constituency, Ranjeet Kamble, on charges of insulting and threatening a gram panchayat (GP) member. PI Bhanudas Pidurkar said a case was lodged under Sections 323, 504 and 506 of IPC against Ranjeet Kamble after receiving a complaint from GP member Ganesh Satav of Senegaon (Bai). Poll Do you support Gurdwara panel's demand to rename Delhi's IGI Airport to Guru Tegh Bahadur Airport? No Yes He said that Senegaon (Bai) sarpanch Ashwini Adkine had filed a complaint against Satav for allegedly insulting Kamble on Thursday during a function in the village. Addressing a press conference, Satav said he questioned Kamble over organizing groundbreaking ceremonies for public works and installing a signboard. “But work is never done. Every citizen should ask such questions to people’s representatives. When I asked Kamble to first perform and stop such publicity, he insulted and threatened me,”

In [41]:

rag_chain = (
    {"context": retriever|format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [42]:

question = "Who is sunil kamble?,give brief summary about him?"
response = rag_chain.invoke(question)

print(response)

1. Sunil Kamble is a BJP MLA (Member of the Legislative Assembly) representing the Cantonment constituency in Pune.
2. He has been booked on charges of slapping a police constable at Sassoon General Hospital in Pune. The incident occurred around 11.30am on Friday, and a case was registered under section 353 (assault or criminal force to deter public servant from discharge of his duty) of the Indian Penal Code.
3. Sunil Kamble is also involved in another allegation, this time in Wardha district, where he has been accused of insulting and threatening a gram panchayat member named Ganesh Satav. A non-cognizable offence was registered against him under Sections 323, 504 and 506 of the Indian Penal Code.
4. Sunil Kamble is currently unavailable for comments as he is busy with a program when contacted by TOI.
5. Sunil Kamble's political affiliation is with the Bharatiya Janata Party (BJP).


In [43]:

question = "give me the positive impressions on sunil kamble?"
response = rag_chain.invoke(question)

print(response)

KeyboardInterrupt: 

In [ ]:
question = "give me the Negative impressions on sunil kamble?"
response = rag_chain.invoke(question)

print(response)

 * Ramesh Bagwe failed to retain his ministerial post in the new cabinet, indicating that there might have been apprehensions or disagreements about his inclusion.
   * Supporters of Bagwe expressed their unhappiness at the turn of events and submitted a memorandum to the district collector for his inclusion in the cabinet.
   * Ramesh Bagwe's brief term as minister was full of controversy, including differences with city police commissioner Satyapal Singh and picking up Rahul Gandhi's shoes in Mumbai.
   * Bagwe expressed dissatisfaction with the functioning of the city Congress, stating that they had not taken corporators and leaders into confidence before supporting changes in the Development Plan.
   * Internal conflict within the Pune Congress party might have been a reason why Ramesh Bagwe and other city Congress MLAs were not included in the state cabinet. This internal conflict may stem from disagreements between different factions, such as those led by Suresh Kalmadi and Vilas

In [44]:
question = "Give me an overview of the a sunil kamble in his political career?"
response = rag_chain.invoke(question)

print(response)

 - Sunil Kamble is a Bharatiya Janata Party (BJP) MLA and incumbent representative for Pune Cantonment Assembly constituency.
   - He has been associated with RSS and ABVP for decades and was elected as a corporator multiple times before his current position.
   - In the political career, he lost the standing committee chairmanship to Yogesh Mulik in previous elections but secured the nomination this time when the NCP and Congress withdrew their candidates.
   - Sunil Kamble is a four-time corporator and has been active in the party for years.
   - He was involved in a controversy where he allegedly slapped a police constable at Sassoon General Hospital, and a case was registered against him under section 353 (assault or criminal force to deter public servant from discharge of his duty) of the Indian Penal Code. The incident happened around 11.30am on Friday.


In [20]:
question = "What is the interesting news happening in Lucknow?"
response = rag_chain.invoke(question)

print(response)

1. The first roadshow for the UP Global Investors' Summit was held in Lucknow on Wednesday, receiving investment proposals of around Rs 76,867 crore from investors both in India and abroad.
2. At least 79 MoUs and expressions of interest were signed during this event.
3. The summit is scheduled to begin on February 10 and is expected to be attended by more than 10,000 people.
4. Notable dignitaries including President Droupadi Murmu and Prime Minister Narendra Modi are expected to attend the summit.
5. Arrangements for these VIP guests and investors, including facilities like movement, seating, and others, have been made at the venue.
6. A food court has been set up adjacent to one of the blocks, along with a VIP lounge and lounges for partner countries, industry, and knowledge partners.
7. The fourth block is being prepared for pre-function activities, the fifth block is for cultural activities, and the sixth block is being prepared for B2B meetings where investment proposals and MoUs

In [45]:
question = "List of the positive keywords associated with sunil kamble and explain each in brief?"
response = rag_chain.invoke(question)

print(response)

1. BJP Member: Sunil Kamble is associated with the Bharatiya Janata Party (BJP).

2. Connections with High-ranking Officials: He is the brother of Dilip Kamble, a BJP MLA and minister of state for social justice and special assistance.

3. Active in the party: Sunil Kamble has been active in the BJP for years.

4. Associated with RSS and ABVP: He has been associated with the Rashtriya Swayamsevak Sangh (RSS) and Akhil Bharatiya Vidyarthi Parishad (ABVP) for decades.

5. Elected Corporator: Sunil Kamble was elected as a corporator earlier, suggesting he has held political office within the Pune Municipal Corporation.

6. Performance-based Nomination: The context suggests that his performance and long association with the party were key factors in his nomination for office.

7. Contesting Elections: Sunil Kamble is also a contestant in elections, running for the seat of MLA (Member of Legislative Assembly) in Pune Cantonment assembly constituency.


In [46]:
question = "List of the Negative keywords associated with sunil kamble and explain each in brief?"
response = rag_chain.invoke(question)

print(response)

 * "Insulting": This refers to the act of showing contempt or disrespect towards someone, as reported in the context that MLA Sunil Kamble allegedly did towards a gram panchayat member named Ganesh Satav.
   * "Threatening": This indicates making someone afraid by using force, violence, or causing harm, as also mentioned in the context where it was alleged that MLA Sunil Kamble threatened Ganesh Satav.
   * "Dadagiri": This term is often used in India to refer to misbehavior, intimidation, or bullying, as can be inferred from the statements made by BJP MP Ramdas Tadas and BJP district unit president Sunil Gafat regarding MLA Sunil Kamble's behavior.
   * "Abusive language": This refers to harsh, disrespectful words or expressions, as reported in the context of the agitation led by women leaders against MLA Sunil Kamble due to his alleged use of such language while speaking with a woman civic official.
   * "Double standards": This term is used when someone consistently applies differen

In [47]:
question = "what can sunil kamble do, to improve campaign outreach?"
response = rag_chain.invoke(question)

print(response)

1. Sunil Kamble can organize bike rallies in different parts of Pune Cantonment assembly constituency, with his supporters leading the way on motorcycles.
2. He can make public appearances at various locations, such as slum areas and upscale neighborhoods, to connect with voters.
3. Sunil Kamble can visit key locations in the constituency, like Bhavani Peth, Nana Peth, and other strategic points.
4. He can appeal to voters while crossing busy intersections, like Sarbatwala Chowk.
5. Kamble can accompany his wife or brother during these outreach initiatives.
6. Sunil Kamble can make speeches at public meetings or rallies, addressing the concerns of the constituents and presenting a progressive vision for the area's development.
7. He can listen to the concerns of slum dwellers patiently, demonstrating accessibility and willingness to help on the ground.
8. Kamble can work towards improving his image by refuting any allegations made against him, approaching the police for investigations 